# Reading in Sentinel-2 Images

Sentinel-2 is an observation mission developed by the European Space Agency to
monitor the surface of the Earth [official website](http://www.esa.int/Our_Activities/Observing_the_Earth/Copernicus/Sentinel-2).
Sets of images are taken of the surface where each image corresponds to a
specific wavelength. These images can provide useful data for a wide variety of
industries, however, the format they are stored can prove difficult to work
with. This being, `JPEG 2000` (file extension `.jp2`), an image compression
format for JPEGs that allow for improved quality and compression ratio.

There are few programs that can work with `jp2`, which can make processing
large amounts of them difficult. Because of GeoPySpark, though, we can leverage
the tools available to us in Python that can work with `jp2` and use them to
format the sentinel data so that it can be ingested.

## Getting the Data

Before we can start this tutorial, we will need to get the sentinel images.
All sentinel data can be found on Amazon's S3, and we will be downloading it
straight from there.

The way the data is stored on S3 will not be discussed here, instead, a general
overview of the data will be given. We will downloading three different `jp2`
that represent the same area and time in different wavelength. These being:
Aerosol detection (443 nm), Water vapor (945 nm), and Cirrus (1375 nm). Why
these three bands? It's because of the resolution of the image, which
determines what bands are represented best. For this example, we will be
working at a 60 m resolution; which provides the best representation of the
mentioned bands. As for what is in the photos, it is the eastern coast of
Corsica taken on January 4th, 2017.

In [ ]:
!curl -o /tmp/B01.jp2 http://sentinel-s2-l1c.s3.amazonaws.com/tiles/32/T/NM/2017/1/4/0/B01.jp2
!curl -o /tmp/B09.jp2 http://sentinel-s2-l1c.s3.amazonaws.com/tiles/32/T/NM/2017/1/4/0/B09.jp2
!curl -o /tmp/B10.jp2 http://sentinel-s2-l1c.s3.amazonaws.com/tiles/32/T/NM/2017/1/4/0/B10.jp2

## The Code

Now that we have the files, we can begin to read them into GeoPySpark.

In [1]:
import rasterio
import numpy as np

from geopyspark import geopyspark_conf
from geopyspark.geotrellis import Tile, Extent, ProjectedExtent
from geopyspark.geotrellis import catalog, layer
from geopyspark.geotrellis.constants import LayerType, LayoutScheme

from pyspark import SparkContext

In [2]:
conf = geopyspark_conf(master="local[*]", appName="ingest-example")
pysc = SparkContext(conf=conf)

## Reading in the JPEG 2000's

`rasterio` being backed by GDAL allows us to read in the `jp2`s.
Because each image represents a wavelength, there is a order in which
they need to be in when they're merged to together into a multiband raster which
is represented by `jp2s`. After the reading process, the list of `numpy`
arrays will be turned into one array. This represents our mulitband raster.

In [3]:
jp2s = ["/tmp/B01.jp2", "/tmp/B09.jp2", "/tmp/B10.jp2"]
arrs = []

for jp2 in jp2s:
    with rasterio.open(jp2) as f:
        arrs.append(f.read(1))

data = np.array(arrs, dtype=arrs[0].dtype)

In [4]:
data.dtype

dtype('uint16')

## Creating the RDD

GeoPySpark is a Python binding of GeoTrellis, and because of that, requires the
data to be in a certain format. Please see
:ref:`core_concepts` to learn what each of these variables represent.

In [5]:
extent = Extent(*f.bounds)
projected_extent = ProjectedExtent(extent=extent, epsg=int(f.crs.to_dict()['init'][5:]))

In [6]:
tile = Tile(cells=data, cell_type='USHORT', no_data_value=f.nodata)

In [7]:
rdd = pysc.parallelize([(projected_extent, tile)])

## Creating the Layer

With our RDD, we can now create a `RasterLayer` using the `from_numpy_rdd` method.

In [8]:
raster_layer = layer.RasterLayer.from_numpy_rdd(pysc=pysc, layer_type=LayerType.SPATIAL, numpy_rdd=rdd)
raster_layer